# Compare posteriors.

This notebook loads up posterior distributions for different shoreline options and visualizes them with corner plots. 

In [ ]:
from shoreline import * 

In [ ]:
import arviz as az 
import corner 

print('Here are the available posteriors:\n')
posteriors = {}
for f in glob.glob('posteriors/*-numpyro.nc'):
    k = f.split('/')[1].split('-numpyro.nc')[0]
    print(f' {k}')
    posteriors[k] = az.from_netcdf(f)
    

Print out parameter values + confidence intervals.

In [ ]:
warnings.simplefilter('ignore')

with open('posteriors-shorelines.tex', 'w') as f:

    for kind, inference in posteriors.items():
        f.write(f'%{kind}\n')
        po = inference.posterior
        po['fo'] = 10**po['log_f_0']
        po['logL_nohz'] = -po['log_f_0']/po['q']
        po['L_nohz'] = 10**po['logL_nohz']
        po['L_nohz'] = 10**po['logL_nohz']
        po['wninetyfive'] = 5.88*po['w']

        po['wninetyfive-as-flux-factor'] = 10**po['wninetyfive']
        po['q-from-ard'] = - po['log_f_0']/(-1.7)

        lines = latexify(po, label=kind.replace('uncertainties', ''))
        f.writelines(lines)

In [ ]:
from scipy.special import erf
levels = erf(np.arange(1,3)/np.sqrt(2))
levels

In [ ]:
plt.close("all")
figsize=(6,6)
question = {"any": "Any atmosphere or surface volatiles?", "CO2": "Warm CO$_2$ atmosphere?"}
better_titles = ["$\sf \log_{10}f_0$", "$\sf p$", "$\sf q$", "$\sf \ln w$"]
var_names = ["log_f_0", "p", "q", "ln_w"] 

kw = dict(
    var_names=var_names,
    labels=better_titles,
    bins=50,
    levels=levels,
    plot_density=False,
    plot_datapoints=False,
    show_titles=True,
    title_kwargs=dict(fontsize=9),
    range=[[-1, 6], [0, 10], [0, 3], [-6, 2]],
)
for kind in ["any", "CO2"]:
    labels = {
        "all": f"Exoplanets + Solar System",
        "exo": "Exoplanets Only",
        "solar": "Solar System Only",
    }
    fig = plt.figure(figsize=figsize)
    colors = {
        f"solar-{kind}": "mediumslateblue",
        f"exo-{kind}": "tomato",
        f"all-{kind}": "black",
    }
    for k in colors:
        if "all" in k:
            alpha = 1
        else:
            alpha = 0.5
        corner.corner(
            posteriors[f'{k}-uncertainties=True'],
            color=colors[k],
            hist_kwargs=dict(density=True, label=labels[k.split("-")[0]]),
            fig=fig,
            contour_kwargs=dict(alpha=alpha, color=colors[k]),
            **kw,
        )
    plt.sca(fig.get_axes()[0])
    plt.legend(bbox_to_anchor=(1, 1), frameon=False)
    plt.sca(fig.get_axes()[2])
    plt.title(question[kind])
    #for ax in fig.get_axes():
    #    ax.title.set_size(10)
        #title = ax.get_title()
        #if title != '':
        #    ax.set_title(title.replace('=', '=\n'))
    plt.savefig(f"figures/posteriors-{kind}.pdf")

In [ ]:
fig = plt.figure(figsize=figsize)
subset = 'all'
for kind in ["CO2", "any"]:
    kind_label = {"any": 'any atmosphere', 
                  "CO2":'CO$_2$ atmosphere'}
    uncertainty_labels = {True:f'with uncertainties', 
                          False:f'without uncertainties'}
    
    colors = {f"any": "green", "CO2": "orchid"}
    # corner.corner(posteriors[f'all-{kind}'], color=colors[kind], hist_kwargs=dict(density=True, label=kind), fig=fig, **kw)
    for u in [True, False]:
        alpha = {True: 1.0, False: 0.3}[u]
        color_kwargs = dict(alpha=alpha, color=colors[kind])
        corner.corner(
            posteriors[f"{subset}-{kind}-uncertainties={u}"],
            color=colors[kind],
            contour_kwargs=color_kwargs,
            hist_kwargs=color_kwargs | dict(density=True, label=f'{kind_label[kind]}, {uncertainty_labels[u]}'),
            fig=fig,
            **(kw | dict(show_titles = False)),
        )
    plt.sca(fig.get_axes()[0])
    plt.legend(bbox_to_anchor=(1, 1), frameon=False)
    plt.sca(fig.get_axes()[0])
    plt.legend(bbox_to_anchor=(1, 1), frameon=False)
    plt.savefig('figures/posteriors-with-and-without-uncertainties.pdf')